<a href="https://colab.research.google.com/github/ShanmukhiKairuppala/Diabetic_Retinopathy_Detection/blob/main/Mini_Project_DR_Flask_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask flask-ngrok scikit-image opencv-python-headless pyngrok

In [ ]:
!ngrok authtoken 2hQVd6MLN11Zzq7SSgrDGxZbTmi_4XsoLQcwPcdSTntXX1fVV


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install scikit-image tensorflow flask-ngrok
!pip install flask flask-ngrok scikit-image opencv-python-headless pyngrok


Mounted at /content/drive


In [ ]:

# import os
# import numpy as np
# import matplotlib.pyplot as plt
# from skimage import io, img_as_ubyte
# import cv2
# from flask import Flask, request, jsonify, send_file
# import tensorflow as tf
# import subprocess
# from pyngrok import ngrok
# import base64

# app = Flask(__name__)
# app.config['UPLOAD_FOLDER'] = 'static/uploads/'

# # Load your model
# model = tf.keras.models.load_model('/content/drive/MyDrive/FineTunedModel')

# def preprocess_image(image, size=(256, 256)):
#     if image.ndim == 2 or image.max() == image.min():
#         print("Skipping image: Image is either grayscale or completely black.")
#         return None

#     if image.ndim == 3 and image.shape[2] == 3:
#         lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
#         l, a, b = cv2.split(lab)
#         clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
#         cl = clahe.apply(l)
#         limg = cv2.merge((cl, a, b))
#         image = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
#     elif image.ndim == 2:
#         clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
#         image = clahe.apply(image)

#     resized_image = cv2.resize(image, size)
#     return img_as_ubyte(resized_image)

# def augment_image(image, base_filename, output_dir, retinopathy=True):
#     transformations = [image]
#     if retinopathy:
#         for angle in [90, 120, 180, 270]:
#             rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
#             transformations.append(rotated_image)
#     mirrored_image = cv2.flip(image, 1)
#     transformations.append(mirrored_image)
#     augmented_filenames = []
#     for idx, transformed_image in enumerate(transformations):
#         transformed_filename = f"{base_filename}_{idx}.png"
#         transformed_path = os.path.join(output_dir, transformed_filename)
#         io.imsave(transformed_path, transformed_image)
#         augmented_filenames.append(transformed_filename)
#     return augmented_filenames

# def preprocess_and_augment_single_image(image_path, output_dir, base_filename):
#     image = io.imread(image_path)
#     preprocessed_image = preprocess_image(image)
#     if preprocessed_image is not None:
#         augmented_filenames = augment_image(preprocessed_image, base_filename, output_dir, True)
#         augmented_images = [io.imread(os.path.join(output_dir, filename)) for filename in augmented_filenames]
#         return preprocessed_image, augmented_images
#     return None, []

# def predict_and_display(model, image_path):
#     temp_dir = '/content/temp'
#     if not os.path.exists(temp_dir):
#         os.makedirs(temp_dir)

#     preprocessed_image, augmented_images = preprocess_and_augment_single_image(image_path, temp_dir, 'new_image')

#     if preprocessed_image is not None:
#         image_array = np.expand_dims(preprocessed_image / 255.0, axis=0)
#         prediction = model.predict(image_array)
#         predicted_class = np.argmax(prediction)
#         dr_levels = ["No DR", "Mild DR", "Moderate DR", "Severe DR", "Proliferative DR"]

#         num_images = 1 + len(augmented_images)
#         plt.figure(figsize=(15, 5))

#         plt.subplot(1, num_images, 1)
#         plt.imshow(preprocessed_image)
#         plt.title('Input Image')

#         for i, augmented_image in enumerate(augmented_images):
#             plt.subplot(1, num_images, i + 2)
#             plt.imshow(augmented_image)
#             plt.title(f'Augmented {i + 1}')

#         plt.suptitle(f'Predicted DR Level: {predicted_class} ({dr_levels[predicted_class]})')
#         result_path = os.path.join(temp_dir, 'result.png')
#         plt.savefig(result_path)
#         plt.close()

#         return result_path, preprocessed_image, augmented_images, predicted_class
#     else:
#         return None, None, [], None

# def encode_image(image):
#     _, buffer = cv2.imencode('.png', image)
#     encoded_image = base64.b64encode(buffer).decode('ascii')
#     return encoded_image

# @app.route('/predict', methods=['POST'])
# def predict():
#     file = request.files['file']
#     temp_file_path = '/content/temp_uploaded_image.png'
#     file.save(temp_file_path)

#     result_path, preprocessed_image, augmented_images, predicted_class = predict_and_display(model, temp_file_path)

#     if result_path is not None:
#         encoded_preprocessed_image = encode_image(preprocessed_image)
#         encoded_augmented_images = [encode_image(img) for img in augmented_images]

#         return jsonify({
#             'prediction': int(predicted_class),
#             'preprocessed_image': encoded_preprocessed_image,
#             'augmented_images': encoded_augmented_images,
#             'result_image_path': result_path
#         })
#     else:
#         return jsonify({'error': 'Image processing failed.'})

# if __name__ == '__main__':
#     # Start ngrok tunnel
#     public_url = ngrok.connect(5000)
#     print(f"Public URL: {public_url}")

#     # Run Flask app
#     app.run()

Mounted at /content/drive
Public URL: NgrokTunnel: "https://3ab5-34-106-96-17.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 [==============================] - 2s 2s/step


INFO:werkzeug:127.0.0.1 - - [12/Jun/2024 03:22:53] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 184ms/step


...

INFO:werkzeug:127.0.0.1 - - [12/Jun/2024 04:32:10] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 163ms/step


INFO:werkzeug:127.0.0.1 - - [12/Jun/2024 04:53:39] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 157ms/step


INFO:werkzeug:127.0.0.1 - - [12/Jun/2024 04:57:54] "POST /predict HTTP/1.1" 200 -


In [ ]:
!pip install imageio[pyav]
!pip install scikit-image
!pip install pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 32.3 MB/s eta 0:00:00


In [ ]:
from flask import Flask, request, jsonify, render_template, send_file
import os
import numpy as np
import cv2
import tensorflow as tf
import base64
from skimage import io, img_as_ubyte
from pyngrok import ngrok
import matplotlib.pyplot as plt
import subprocess



app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'static/uploads/'

# Load your model
model = tf.keras.models.load_model('/content/drive/MyDrive/FineTunedModel')

# Detailed information for each DR level
dr_info = {
    0: {
        "meaning": "No signs of diabetic retinopathy. The retina appears normal with no abnormalities present.",
        "presence": "Normal retina with no abnormalities.",
        "causes": "Good management of diabetes and overall health.",
        "consequences": "Maintaining this level ensures healthy vision and prevents the onset of diabetic retinopathy or other eye diseases such as glaucoma and cataracts.",
        "precautions": "Maintain good control of your blood sugar, blood pressure, and cholesterol. Have regular eye exams to monitor for any changes.",
        "diet": "Balanced diet with a focus on managing blood sugar levels. Foods to eat: vegetables, fruits, whole grains, lean proteins, and healthy fats. Foods to avoid: sugary foods, high-carb processed foods, and high-fat foods.",
        "exercise": "Regular physical activity to manage diabetes. Recommended exercises: brisk walking, cycling, swimming, and light jogging.",
        "doctor": "Regular follow-ups with your healthcare provider."
    },
    1: {
        "meaning": "Small areas of swelling in the retina's tiny blood vessels, known as microaneurysms. These are the earliest visible signs of diabetic retinopathy.",
        "presence": "Microaneurysms or small bulges in the blood vessels.",
        "causes": "Early effects of diabetes on the retina.",
        "consequences": "May progress if diabetes is not well managed, potentially leading to more severe diabetic retinopathy or macular edema.",
        "precautions": "Continue regular monitoring and maintain good control of your diabetes. Follow up with your eye care professional as recommended.",
        "diet": "Low sugar and low-fat diet. Foods to eat: non-starchy vegetables, lean proteins, whole grains, and low-fat dairy. Foods to avoid: sugary snacks, high-fat meats, and refined grains.",
        "exercise": "Consistent exercise to keep diabetes under control. Recommended exercises: walking, yoga, water aerobics, and cycling.",
        "doctor": "Regular check-ups and follow the treatment plan."
    },
    2: {
        "meaning": "Blood vessels that nourish the retina are blocked, leading to more significant damage, such as microaneurysms, hemorrhages (bleeding), and hard exudates (fatty deposits).",
        "presence": "Microaneurysms, retinal hemorrhages, and hard exudates.",
        "causes": "Progression of diabetes affecting retinal blood vessels.",
        "consequences": "Increased risk of vision impairment if untreated, as well as potential for diabetic macular edema or retinal vein occlusion.",
        "precautions": "Increase the frequency of eye exams. Follow your doctor's advice to manage your diabetes more effectively.",
        "diet": "Strict diabetic diet with controlled carbohydrate intake. Foods to eat: high-fiber foods, lean proteins, and healthy fats. Foods to avoid: high-sugar foods, refined carbohydrates, and high-sodium foods.",
        "exercise": "Increased physical activity as recommended by your doctor. Recommended exercises: moderate-intensity aerobic exercises, strength training, and flexibility exercises.",
        "doctor": "More frequent monitoring and possible medication adjustments."
       },
    3: {
       "meaning": "Many blood vessels are blocked, depriving areas of the retina of their blood supply, leading to significant retinal damage including hemorrhages, hard and soft exudates (cotton wool spots), and more widespread microaneurysms.",
        "presence": "Significant blood vessel blockage, retinal hemorrhages, hard exudates, soft exudates (cotton wool spots), and extensive microaneurysms.",
        "causes": "Advanced effects of poorly managed diabetes.",
        "consequences": "High risk of severe vision loss or blindness if untreated. Other complications can include retinal detachment and neovascular glaucoma.",
        "precautions": "Frequent monitoring and possible treatment to prevent progression. Follow medical advice closely.",
        "diet": "Very strict diabetic diet and possibly medical nutrition therapy. Foods to eat: high-fiber vegetables, lean proteins, low-glycemic index foods. Foods to avoid: sugary foods, high-carb foods, and trans fats.",
        "exercise": "Tailored exercise program under medical supervision. Recommended exercises: low-impact activities such as walking, stationary cycling, and swimming.",
        "doctor": "Close supervision by an ophthalmologist and diabetes specialist."

       },
    4: {
        "meaning": "The most advanced stage with new blood vessels growing on the retina (neovascularization), which can leak and cause severe vision problems such as vitreous hemorrhage or retinal detachment. Presence of microaneurysms, hemorrhages, hard exudates, soft exudates (cotton wool spots), and proliferative changes.",
        "presence": "Proliferation of abnormal blood vessels, extensive microaneurysms, retinal hemorrhages, hard exudates, soft exudates (cotton wool spots).",
        "causes": "Severe and prolonged high blood sugar levels.",
        "consequences": "High risk of severe vision impairment or blindness. Other complications include tractional retinal detachment and neovascular glaucoma.",
        "precautions": "Immediate treatment is required to prevent severe vision loss. Treatments may include laser therapy, injections, or surgery.",
        "diet": "Strict diabetic diet, possibly involving a dietitian. Foods to eat: non-starchy vegetables, high-fiber foods, lean proteins, and healthy fats. Foods to avoid: sugary foods, high-carb foods, processed foods, and high-sodium foods.",
        "exercise": "Exercise under strict medical guidance. Recommended exercises: low-impact activities such as walking and stationary cycling, avoiding any activity that can cause strain or increase blood pressure significantly.",
        "doctor": "Immediate intervention by an ophthalmologist, with possible surgical treatments."

        }
}

def preprocess_image(image, size=(256, 256)):
    if image.ndim == 2 or image.max() == image.min():
        print("Skipping image: Image is either grayscale or completely black.")
        return None

    if image.ndim == 3 and image.shape[2] == 3:
        lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        cl = clahe.apply(l)
        limg = cv2.merge((cl, a, b))
        image = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
    elif image.ndim == 2:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        image = clahe.apply(image)

    resized_image = cv2.resize(image, size)
    return img_as_ubyte(resized_image)

def augment_image(image, base_filename, output_dir, retinopathy=True):
    transformations = [image]
    if retinopathy:
        for angle in [90, 120, 180, 270]:
            rotated_image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
            transformations.append(rotated_image)
    mirrored_image = cv2.flip(image, 1)
    transformations.append(mirrored_image)
    augmented_filenames = []
    for idx, transformed_image in enumerate(transformations):
        transformed_filename = f"{base_filename}_{idx}.png"
        transformed_path = os.path.join(output_dir, transformed_filename)
        io.imsave(transformed_path, transformed_image)
        augmented_filenames.append(transformed_filename)
    return augmented_filenames

def preprocess_and_augment_single_image(image_path, output_dir, base_filename):
    image = io.imread(image_path)
    preprocessed_image = preprocess_image(image)
    if preprocessed_image is not None:
        augmented_filenames = augment_image(preprocessed_image, base_filename, output_dir, True)
        augmented_images = [io.imread(os.path.join(output_dir, filename)) for filename in augmented_filenames]
        return preprocessed_image, augmented_images
    return None, []

def predict_and_display(model, image_path):
    temp_dir = '/content/temp'
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)

    preprocessed_image, augmented_images = preprocess_and_augment_single_image(image_path, temp_dir, 'new_image')

    if preprocessed_image is not None:
        image_array = np.expand_dims(preprocessed_image / 255.0, axis=0)
        prediction = model.predict(image_array)
        predicted_class = np.argmax(prediction)

        num_images = 1 + len(augmented_images)
        plt.figure(figsize=(15, 5))

        plt.subplot(1, num_images, 1)
        plt.imshow(preprocessed_image)
        plt.title('Input Image')

        for i, augmented_image in enumerate(augmented_images):
            plt.subplot(1, num_images, i + 2)
            plt.imshow(augmented_image)
            plt.title(f'Augmented {i + 1}')

        plt.suptitle(f'Predicted DR Level: {predicted_class} ({dr_info[predicted_class]["meaning"]})')
        result_path = os.path.join(temp_dir, 'result.png')
        plt.savefig(result_path)
        plt.close()

        return result_path, preprocessed_image, augmented_images, predicted_class
    else:
        return None, None, [], None

def encode_image(image):
    _, buffer = cv2.imencode('.png', image)
    encoded_image = base64.b64encode(buffer).decode('ascii')
    return encoded_image

@app.route('/predict', methods=['POST'])
def predict():
    file = request.files['file']
    temp_file_path = '/content/temp_uploaded_image.png'
    file.save(temp_file_path)

    result_path, preprocessed_image, augmented_images, predicted_class = predict_and_display(model, temp_file_path)

    if result_path is not None:
        encoded_preprocessed_image = encode_image(preprocessed_image)
        encoded_augmented_images = [encode_image(img) for img in augmented_images]

        return jsonify({
            'prediction': int(predicted_class),
            'preprocessed_image': encoded_preprocessed_image,
            'augmented_images': encoded_augmented_images,
            'result_image_path': result_path,
            'dr_info': dr_info[predicted_class]
        })
    else:
        return jsonify({'error': 'Image processing failed.'})

if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")
    app.run()


Public URL: NgrokTunnel: "https://326d-34-23-218-215.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 [==============================] - 3s 3s/step


INFO:werkzeug:127.0.0.1 - - [13/Jun/2024 16:27:04] "POST /predict HTTP/1.1" 200 -
